In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)
template= PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}"
)

prompt = template.format(country_a="Mexico", country_b="Thailand")



chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [ ]:
template =ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ('ai', "Ciao, mi chiamo {name}!"),
    ('human', "What is the distance between {country_a} and {country_b}. Alos what is your name?")
])

prompt = template.format_messages(
    language='Greek',
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')